In [8]:
import requests
from alphacast import Alphacast
import pandas as pd
import time
from clave_api_alphacast import api_key

In [9]:
alphacast = Alphacast(api_key)

In [10]:
tc_serie_alpha = alphacast.datasets.dataset(5288).download_data("pandas")
tc_serie_alpha

,Date,country,ADR GGAL,BLUE,Dolar CCL,Dolar MEP,Dolar Mayorista,Dolar Oficial,GGAL,Merval,...,Brecha CCL - 30d_change,Brecha Blue - 30d_change,Dolar Oficial - 30d_change,Dolar Solidario,Dolar Solidario - 30d_running_av,Dolar Solidario - 30d_running_av_mom_change,Dolar Solidario - 30d_change,Dolar Solidario - usd_daily,Dolar Solidario - 7d_running_av,Dolar Solidario - 1d_change
0,1992-01-02,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-01-03,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-01-06,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-01-07,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-01-08,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,2023-10-05,Argentina,12.71,838.0,893.88,784.83,350.025,367.090,1126.0,614300.92,...,0.321552,0.318696,0.000218,639.625,639.625000,0.046279,0.000000,1.827448,639.625000,0.000000
8495,2023-10-06,Argentina,12.82,875.0,891.96,812.75,350.025,367.020,1104.8,635267.38,...,0.312193,0.438185,0.000027,639.625,639.625000,0.039090,0.000000,1.827500,639.625000,0.000000
8496,2023-10-09,Argentina,12.37,940.0,881.52,840.01,350.025,367.180,1083.4,652993.60,...,0.358728,0.661547,0.000463,639.625,639.625000,0.018673,0.000000,1.829016,639.625000,0.000000
8497,2023-10-10,Argentina,12.03,1000.0,953.56,844.81,350.025,367.180,1147.1,713726.09,...,0.542893,0.830525,0.000463,731.000,642.670833,0.017193,0.142857,2.088870,652.678571,14.285714


In [11]:
df = pd.read_excel('../output/ITCRM historico.xlsx',sheet_name="tc_equilibrio")
tc_read_df = pd.read_excel('../output/Tc a precios de hoy.xlsx')[['Período','dolar','inflacion','tc_real_hoy']]

In [12]:
tc_serie_alpha = tc_serie_alpha.set_index("Date")
tc_serie_alpha.index = pd.to_datetime(tc_serie_alpha.index, format = "%Y-%m-%d")
tc_serie_alpha = tc_serie_alpha[["BLUE", "Dolar CCL", "Dolar MEP", "Dolar Solidario"]]
df = df.set_index("Período")

In [13]:
df_brechas = df.merge(tc_serie_alpha, how = "outer", left_index = True, right_index = True)
df_brechas["blue-equilibrio"] = (df_brechas["BLUE"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["ccl-equilibrio"] = (df_brechas["Dolar CCL"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["mep-equilibrio"] = (df_brechas["Dolar MEP"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["solidario-equilibrio"] = (df_brechas["Dolar Solidario"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]

In [14]:
df

,tc_equilibrio,tc_oficial_mayorista,brecha
Período,,,
1997-11-24,1.924512,1.0,0.924512
1997-11-25,1.927250,1.0,0.927250
1997-11-26,1.932110,1.0,0.932110
1997-11-27,1.933234,1.0,0.933234
1997-11-28,1.932836,1.0,0.932836
...,...,...,...
2023-10-06,606.278879,350.1,0.731731
2023-10-07,607.952845,350.1,0.736512
2023-10-08,609.626690,350.1,0.741293


In [15]:
df_brechas

,tc_equilibrio,tc_oficial_mayorista,brecha,BLUE,Dolar CCL,Dolar MEP,Dolar Solidario,blue-equilibrio,ccl-equilibrio,mep-equilibrio,solidario-equilibrio
1992-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-07,607.952845,350.1,0.736512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-08,609.626690,350.1,0.741293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-09,610.651474,350.1,0.744220,940.0,881.52,840.01,639.625,0.53934,0.443573,0.375596,0.047447
2023-10-10,610.277251,350.1,0.743151,1000.0,953.56,844.81,731.000,0.63860,0.562503,0.384305,0.197816


In [23]:
df_brechas.index[-1].strftime("%d/%m/%Y")

'11/10/2023'

In [26]:
import plotly.graph_objects as go

fig = go.Figure()
df_brechas = df_brechas["2011":]
ultima_fecha = df_brechas.index[-1].strftime("%d/%m/%Y")
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["solidario-equilibrio"], name = "Solidario"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["blue-equilibrio"], name = "Blue"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["ccl-equilibrio"], name = "CCL"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["mep-equilibrio"], name = "MEP"))
fig.update_yaxes(tickformat = ".2%")
fig.update_layout(template = None, 
                  title_text = f"Brecha entre el tc de equilibrio y las distintas cotizaciones<br><sup>Datos hasta {ultima_fecha}")


In [17]:
# alphacast.datasets.dataset(37268).download_data("pandas")
alphacast.datasets.dataset(37268).upload_data_from_df(df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)
time.sleep(10)
alphacast.datasets.dataset(37447).upload_data_from_df(tc_read_df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)

b'{"id": 2992800, "status": "Requested", "createdAt": "2023-10-12T13:45:06.437555", "datasetId": 37447}'

In [7]:
time.sleep(3)
alphacast.datasets.dataset(37268).processes()


b'[{"id": 2896563, "datasetId": 37268, "status": "Processed", "statusDescription": "456 values had conflict and were updated./n9398 values where missing in data, and were deleted from database./n31 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-09-18T23:42:17", "processedAt": "2023-09-18T23:42:19"}, {"id": 2773809, "datasetId": 37268, "status": "Processed", "statusDescription": "322 values had conflict and were updated./n37 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-08-18T11:47:26", "processedAt": "2023-08-18T11:47:29"}, {"id": 2626264, "datasetId": 37268, "status": "Processed", "statusDescription": "338 values had conflict and were updated./n57 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-07-13T13:49:15", "processedAt": "2023-07-13T13:49:17"}, {"id": 2327778, "datasetId": 37268, "status": "Processed", "statusDescription": 

In [18]:
time.sleep(10)
alphacast.datasets.dataset(37447).processes()

b'[{"id": 2992800, "datasetId": 37447, "status": "Processed", "statusDescription": "0 values added to database./nDataset has not changed. Avoiding csv version upload.", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-10-12T13:45:06", "processedAt": "2023-10-12T13:45:09"}, {"id": 2896564, "datasetId": 37447, "status": "Processed", "statusDescription": "520 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n1 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-09-18T23:42:28", "processedAt": "2023-09-18T23:42:30"}, {"id": 2773810, "datasetId": 37447, "status": "Processed", "statusDescription": "516 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n2 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-08-18T11:47:37", "proce

In [19]:
# time.sleep(10)
# alphacast.datasets.dataset(37268).download_data("pandas")